# Assignment 3
## Econ 8310 - Business Forecasting

For homework assignment 3, you will work with [Fashion MNIST](https://github.com/zalandoresearch/fashion-mnist), a more fancier data set.

- You must create a custom data loader as described in the first week of neural network lectures [2 points]
    - You will NOT receive credit for this if you use the pytorch prebuilt loader for Fashion MNIST!
- You must create a working and trained neural network using only pytorch [2 points]
- You must store your weights and create an import script so that I can evaluate your model without training it [2 points]

Highest accuracy score gets some extra credit!

Submit your forked repository URL on Canvas! :) I'll be manually grading this assignment.

Some checks you can make on your own:
- Did you manually process the data or use a prebuilt loader (see above)?
- Does your script train a neural network on the assigned data?
- Did your script save your model?
- Do you have separate code to import your model for use after training?

In [26]:
import pandas as pd
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import plotly.express as px
import torch
import torch.nn as nn
import torch.nn.functional as F
import urllib.request
import gzip
import numpy as np
import struct

# Custom dataloader for the Fashion-MNIST dataset
class FashionMNIST(Dataset):
    def __init__(self, train_images_url, train_labels_url, test_images_url, test_labels_url, train=True):
        self.train = train
        self.unzip(train_images_url, train_labels_url, test_images_url, test_labels_url)

    # Credit to Google and its AI search assistant as my source for this section on how to unzip and use the .gz files for this custom dataloader
    def unzip(self, train_images_url, train_labels_url, test_images_url, test_labels_url):
        with urllib.request.urlopen(train_images_url) as response, gzip.GzipFile(fileobj=response) as f:
            magic, num_images, rows, cols = struct.unpack(">IIII", f.read(16))
            self.train_images = np.frombuffer(f.read(), dtype=np.uint8).reshape(num_images, rows, cols)

        with urllib.request.urlopen(train_labels_url) as response, gzip.GzipFile(fileobj=response) as f:
            magic, num_labels = struct.unpack(">II", f.read(8))
            self.train_labels = np.frombuffer(f.read(), dtype=np.uint8).reshape(num_labels)

        with urllib.request.urlopen(test_images_url) as response, gzip.GzipFile(fileobj=response) as f:
            magic, num_images, rows, cols = struct.unpack(">IIII", f.read(16))
            self.test_images = np.frombuffer(f.read(), dtype=np.uint8).reshape(num_images, rows, cols)

        with urllib.request.urlopen(test_labels_url) as response, gzip.GzipFile(fileobj=response) as f:
            magic, num_labels = struct.unpack(">II", f.read(8))
            self.test_labels = np.frombuffer(f.read(), dtype=np.uint8).reshape(num_labels)

    def __len__(self):
        return len(self.train_labels) if self.train else len(self.test_labels)

    def __getitem__(self, idx):
        if self.train:
            image = torch.tensor(self.train_images[idx], dtype=torch.float32).unsqueeze(0)
            label = torch.tensor(self.train_labels[idx], dtype=torch.long)
        else:
            image = torch.tensor(self.test_images[idx], dtype=torch.float32).unsqueeze(0)
            label = torch.tensor(self.test_labels[idx], dtype=torch.long)

        return image, label

# URLs for training
train_images_url = 'https://github.com/zalandoresearch/fashion-mnist/raw/b2617bb6d3ffa2e429640350f613e3291e10b141/data/fashion/train-images-idx3-ubyte.gz'
train_labels_url = 'https://github.com/zalandoresearch/fashion-mnist/raw/b2617bb6d3ffa2e429640350f613e3291e10b141/data/fashion/train-labels-idx1-ubyte.gz'

# URLs for testing
test_images_url = 'https://github.com/zalandoresearch/fashion-mnist/raw/b2617bb6d3ffa2e429640350f613e3291e10b141/data/fashion/t10k-images-idx3-ubyte.gz'
test_labels_url = 'https://github.com/zalandoresearch/fashion-mnist/raw/b2617bb6d3ffa2e429640350f613e3291e10b141/data/fashion/t10k-labels-idx1-ubyte.gz'

# Create an instances for both the training dataset and testing dataset
train_dataset = FashionMNIST(train_images_url, train_labels_url, test_images_url, test_labels_url, train=True)
test_dataset = FashionMNIST(train_images_url, train_labels_url, test_images_url, test_labels_url, train=False)

# Create the dataLoaders
train_dataloader = DataLoader(train_dataset, batch_size=64)
test_dataloader = DataLoader(test_dataset, batch_size=64)

# len(train_dataset)
# len(test_dataset)

# idx=2
# print(f"This image is labeled a {train_dataset.__getitemTrain__(idx)[1]}")
# px.imshow(train_dataset.__getitemTrain__(idx)[0].reshape(28, 28))

# idx=2
# print(f"This image is labeled a {train_dataset.__getitemTest__(idx)[1]}")
# px.imshow(test_dataset.__getitemTest__(idx)[0].reshape(28, 28))

In [21]:
class FashionNet(nn.Module):
    def __init__(self):
      super(FashionNet, self).__init__()
      self.flatten = nn.Flatten()

      self.linear_relu_model = nn.Sequential(
            nn.LazyLinear(10),
        )

    def forward(self, x):
      x = self.flatten(x)
      output = self.linear_relu_model(x)
      return output

model = FashionNet()

In [22]:
# Define some training parameters
learning_rate = 1e-2
batch_size = 64
epochs = 20

# Define our loss function
#   This one works for multiclass problems
loss_fn = nn.CrossEntropyLoss()

# Build our optimizer with the parameters from
#   the model we defined, and the learning rate
#   that we picked
optimizer = torch.optim.SGD(model.parameters(),
     lr=learning_rate)

In [23]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode
    # important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    # Loop over batches via the dataloader
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation and looking for improved gradients
        loss.backward()
        optimizer.step()
        # Zeroing out the gradient (otherwise they are summed)
        #   in preparation for next round
        optimizer.zero_grad()

        # Print progress update every few loops
        if batch % 10 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [24]:
def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode
    # important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures
    # that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations
    # and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    # Printing some output after a testing round
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [27]:
# Train the model
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 116.636414  [   64/60000]
loss: 3874.153076  [  704/60000]
loss: 6645.219727  [ 1344/60000]
loss: 1359.909302  [ 1984/60000]
loss: 4790.759766  [ 2624/60000]
loss: 2746.125000  [ 3264/60000]
loss: 1421.954468  [ 3904/60000]
loss: 2783.031494  [ 4544/60000]
loss: 3056.596436  [ 5184/60000]
loss: 4960.110840  [ 5824/60000]
loss: 700.722656  [ 6464/60000]
loss: 1035.757324  [ 7104/60000]
loss: 2515.841309  [ 7744/60000]
loss: 3583.726562  [ 8384/60000]
loss: 2051.877930  [ 9024/60000]
loss: 5048.052246  [ 9664/60000]
loss: 761.846741  [10304/60000]
loss: 3646.296631  [10944/60000]
loss: 1610.748657  [11584/60000]
loss: 6676.822754  [12224/60000]
loss: 1157.899170  [12864/60000]
loss: 2123.333984  [13504/60000]
loss: 1851.886353  [14144/60000]
loss: 827.585754  [14784/60000]
loss: 1341.860352  [15424/60000]
loss: 1408.541504  [16064/60000]
loss: 1297.861084  [16704/60000]
loss: 1588.118774  [17344/60000]
loss: 1342.023193  [17984/60000]
loss: 1

In [28]:
# Save our model for later, so we can train more or make predictions

EPOCH = epochs
# We use the .pt file extension by convention for saving
#    pytorch models
PATH = "model.pt"

# The save function creates a binary storing all our data for us
torch.save({
            'epoch': EPOCH,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            }, PATH)

In [29]:
# Specify our path
PATH = "model.pt"

# Create a new "blank" model to load our information into
model = FashionNet()

# Recreate our optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Load back all of our data from the file
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
EPOCH = checkpoint['epoch']